In [104]:
# !pip install dico/fr_core_news_md-3.7.0-py3-none-any.whl
# !pip install pandas
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

In [105]:
import pandas as pd
import re
import spacy
import os
from urllib.parse import urlparse
from fuzzywuzzy import fuzz

# Charger le modèle spaCy
nlp = spacy.load("fr_core_news_md")

import platform
if platform.system() == 'Windows':
    folder = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Scrapping\\Datasets\\Juin'
else:
    folder = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Scrapping/Datasets/Juin'

# Fonction pour obtenir tous les fichiers CSV dans un dossier et ses sous-dossiers
def get_all_csv_files(folder):
    csv_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    print("Les traitements sont en cours, cela pourrait prendre quelques minutes...")
    return csv_files

# Obtenir la liste de tous les fichiers CSV
file_paths = get_all_csv_files(folder)

# Charger et fusionner les fichiers
df_list = [pd.read_csv(file) for file in file_paths]
df = pd.concat(df_list, ignore_index=True)


# Convertir toutes les valeurs en minuscules pour les colonnes de type str, sauf pour les colonnes spécifiées
columns_to_exclude = ['superficie', 'nb_pieces', 'nb_salle_de_bain', 'scraping_date', 'link']
df = df.apply(lambda col: col.str.lower() if col.name not in columns_to_exclude and col.dtype == 'object' else col)

# Filtrer les lignes en fonction de la longueur de la description
seuil_longueur_description = 200
df = df[df['description'].str.len() >= seuil_longueur_description]

# Fonction pour extraire le nom de domaine à partir de l'URL
def extract_site(link):
    try:
        return urlparse(link).netloc
    except:
        return None

# Ajouter la colonne 'site' en extrayant le nom de domaine de la colonne 'link'
if 'link' in df.columns:
    df['site'] = df['link'].apply(extract_site)


# Fonction pour extraire la date à partir de 'scraping_date'
def extract_date(scraping_date):
    try:
        return scraping_date.split(' ')[0]
    except:
        return None

# Ajouter la colonne 'date' en extrayant la date de 'scraping_date'
if 'scraping_date' in df.columns:
    df['date'] = df['scraping_date'].apply(extract_date)

# Convertir la colonne 'date' au format datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')


# Supprimer les lignes où 'price' contient des indications de paiement journalier
df = df[~df['price'].str.contains(r'par jour|/ jour', case=False, na=False)]

# Supprimer les lignes avec des prix manquants
df = df.dropna(subset=['price'])


# Fonction pour nettoyer et convertir les valeurs de 'price' en format numérique
def clean_price(price):
    # Extraire les chiffres de la chaîne de caractères
    price = re.sub(r'[^\d]', '', str(price))
    # Convertir en entier
    return float(price) if price.isdigit() else None

# Appliquer la fonction de nettoyage sur la colonne 'price'
df['price'] = df['price'].apply(clean_price)

# Fonction pour nettoyer et convertir les valeurs de 'superficie' en format float
def clean_superficie(superficie):
    # Retirer toutes les occurrences de "m2" et extraire les chiffres
    superficie = re.sub(r'\s*m2\s*', '', str(superficie), flags=re.IGNORECASE)
    # Convertir en float
    try:
        return float(superficie)
    except ValueError:
        return None

# Appliquer la fonction de nettoyage sur la colonne 'superficie' et renommer la colonne
if 'superficie' in df.columns:
    df['superficie_m2'] = df['superficie'].apply(clean_superficie)
    df.drop(columns=['superficie'], inplace=True)

# Fonction pour nettoyer la colonne 'nb_salle_de_bain'
def clean_nb_salle_de_bain(nb_salle_de_bain):
    if pd.isna(nb_salle_de_bain):
        return None
    # Utiliser une expression régulière pour extraire le nombre
    match = re.search(r'\d+', nb_salle_de_bain)
    if match:
        return int(match.group())
    return None

# Appliquer la fonction à la colonne 'nb_salle_de_bain'
df['nb_salle_de_bain'] = df['nb_salle_de_bain'].apply(clean_nb_salle_de_bain)

# Fonction pour gérer les doublons primaires
def remove_primary_duplicates(df):
    primary_columns = [col for col in df.columns if col not in ['scraping_date', 'date']]
    duplicates = df[df.duplicated(subset=primary_columns, keep=False)]
    primary_duplicates_count = duplicates.shape[0]
    print(f"Nombre de doublons primaires détectés: {primary_duplicates_count}")
    df = df.drop_duplicates(subset=primary_columns, keep='first')
    return df, duplicates

# Appliquer la suppression des doublons primaires
df, primary_duplicates = remove_primary_duplicates(df)


# Fonction pour gérer les doublons secondaires
def remove_secondary_duplicates(df):
    df['description_clean'] = df['description'].apply(lambda x: re.sub(r'\W+', ' ', str(x)))
    duplicates = set()
    seen = set()

    for i, row in df.iterrows():
        if i in seen:
            continue
        for j, other_row in df.iterrows():
            if i != j and j not in seen:
                similarity = fuzz.token_set_ratio(row['description_clean'], other_row['description_clean'])
                if similarity > 98:
                    duplicates.add(j)
                    seen.add(j)
                    break  # Exit the inner loop to ensure only one duplicate is removed

    secondary_duplicates_count = len(duplicates)
    print(f"Nombre de doublons secondaires détectés: {secondary_duplicates_count}")

    duplicates_list = list(duplicates)
    secondary_duplicates = df.loc[duplicates_list]
    df = df.drop(duplicates_list)
    df.drop(columns=['description_clean'], inplace=True)
    return df, secondary_duplicates

# Appliquer la suppression des doublons secondaires
df, secondary_duplicates = remove_secondary_duplicates(df)


# Fusionner les doublons primaires et secondaires
all_duplicates = pd.concat([primary_duplicates, secondary_duplicates]).drop_duplicates()

# # Exporter le dataframe après la suppression des doublons
# import platform
# if platform.system() == 'Windows':
#     output_path_after = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Traitements\\Datasets\\trying_data_after_deduplication.csv'
# else:
#     output_path_after = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Traitements/Datasets/trying_data_after_deduplication.csv'

# df.to_csv(output_path_after, index=False)

# # Exporter les doublons supprimés
# output_path_duplicates = r'D:\Bureau\MemoiresStages\Travaux_techniques\Traitements\Datasets\trying_data_duplicates.csv'
# all_duplicates.to_csv(output_path_duplicates, index=False)

# print(f"Fichier après suppression des doublons sauvegardé sous {output_path_after}")
# print(f"Doublons supprimés sauvegardés sous {output_path_duplicates}")



# Ajouter une colonne 'nb_pieces' en extrayant le nombre de pièces du titre
def extract_nb_pieces(title):
    match = re.search(r'\b(\d+)\s*(pi[eè]ces?|pcs?)\b', title)
    if match:
        return int(match.group(1))
    if "studio" in title or "chambre salon" in title:
        return 1
    return None

if 'title' in df.columns:
    df['nb_pieces'] = df['title'].apply(extract_nb_pieces)

# Convertir 'nb_pieces' en int
df['nb_pieces'] = df['nb_pieces'].astype('Int64')

# Détection et suppression des lignes évoquant la vente, entrepôt ou avec un prix > 10000000
vente_keywords = ['vente', 'vendre', 'à vendre', 'a vendre', 'prix de vente', 'coût de vente', 'cout de vente']
entrepot_keywords = ['entrepôt', 'entrepot', 'hotel', 'hôtel', 'saisonnière', 'saisonniere']

def is_for_sale_or_entrepot(description, price):
    if isinstance(description, str) and (any(keyword in description for keyword in vente_keywords) or any(keyword in description for keyword in entrepot_keywords)):
        return True
    if price and price > 10000000:
        return True
    return False

df = df[~df.apply(lambda row: is_for_sale_or_entrepot(row['description'], row['price']), axis=1)]


# Supprimer les lignes contenant plusieurs biens immobiliers dans le titre
multiple_properties_regex1 = r'appartements|studios|maisons|villas|bureaux'
df = df[~df['title'].str.contains(multiple_properties_regex1, case=False, na=False)]

# Supprimer les lignes contenant plusieurs biens immobiliers dans le titre
multiple_properties_regex2 = r'appartements?[^,]*,|appartements?[^et]*et|maisons?,|bureaux?,|studios?,|villas?,'
df = df[~df['title'].str.contains(multiple_properties_regex2, case=False, na=False)]

# Supprimer les lignes contenant "et" dans le titre
df = df[~df['title'].str.contains(r'\bet\b', case=False, na=False)]

# Supprimer les lignes contenant "terrain" dans le titre
df = df[~df['title'].str.contains(r'\bterrain\b', case=False, na=False)]


def correct_price_from_description(description, price):
    if not isinstance(description, str):
        return price

    # Suppression des lignes avec paiement journalier dans la description
    if re.search(r'\b(par jours?|/ jours?|parjours?|/jours?|/ 1 jours?|/1 jours?|la nuitée|la nuitee|la nuit|par nuit|jour)\b', description):
        return None

    # List of regex patterns to match different price formats
    patterns = [
        r'\b(loyer|prix|prix location|cout location|location|cout|coût)\s*[:\s]*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bà\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bloyer\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bcout\s*[:\s]*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bprix\s*[:\s]*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\b(loyer|prix)\s*=\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\b(loyer|prix|location)\s*[:\s]*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa)?(?!.*frais)',
        r'\bà\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\b(loyer|prix)\s*[:\s]*([\d\s]+)\s*(f|fcfa|francs|cfa|francs cfa|cfa)?',
        r'\b(loyer)\s*:\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bloyer\s*:\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bà\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bloyer\s*[:\s]*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bà\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\bloyer\s*[:\s]*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?',
        r'\b(loyer|prix|location)\s*:\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa)?\s*x?\s*[\d]*(?!.*frais)',
        r'\b(loyer|prix)\s*[:\s]*([\d\s]+)\s*(f|fcfa|francs|cfa|francs cfa|cfa)?',
        r'\bloyer\s*:\s*([\d\s]+)\s*(mille|frs|fcfa|f|francs|f cfa|cfa|fcfa/mois)?'
    ]

    for pattern in patterns:
        match = re.search(pattern, description)
        if match:
            price_str = re.sub(r'\s+', '', match.group(2) if len(match.groups()) > 1 and match.group(2) else match.group(1))
            if price_str.isdigit():
                corrected_price = int(price_str)
                if len(match.groups()) > 2 and match.group(3) and 'mille' in match.group(3):
                    corrected_price *= 1000
                return corrected_price
    return price

# Supprimer les lignes avec des descriptions manquantes ou 'None'
df = df.dropna(subset=['description'])

# Appliquer la correction des prix sur la colonne 'description' et supprimer les lignes contenant 'par jour'
df['price'] = df.apply(lambda row: correct_price_from_description(row['description'], row['price']), axis=1)
df = df.dropna(subset=['price'])

# Supprimer les lignes avec un prix inférieur à 40000
df = df[df['price'] > 39999]


def supprimer_lignes_nb_pieces_manquant(df):
    # Définir le motif regex
    motif = re.compile(r'\b(\d+)\s*(pi[eè]ces?|pcs?)\b')

    def garder_ligne(row):
        # Vérifier si 'nb_pieces' est manquant
        if pd.isna(row['nb_pieces']):
            # Vérifier si le motif n'est pas trouvé dans la description
            if not motif.search(str(row['description'])):
                return False
        return True

    # Appliquer la fonction pour filtrer les lignes
    df = df[df.apply(garder_ligne, axis=1)]
    return df

# Appliquer la fonction au dataframe
df = supprimer_lignes_nb_pieces_manquant(df)

def imputer_nb_pieces(df):
    # Définir le motif regex pour trouver le nombre de pièces
    motif = re.compile(r'\b(\d+)\s*(pi[eè]ces?|pcs?)\b')

    def imputer_ligne(row):
        # Vérifier si 'nb_pieces' est manquant
        if pd.isna(row['nb_pieces']):
            # Rechercher le motif dans la description
            match = motif.search(str(row['description']))
            if match:
                # Extraire et retourner le nombre de pièces
                row['nb_pieces'] = int(match.group(1))
        return row

    # Appliquer la fonction pour imputer les valeurs manquantes de 'nb_pieces'
    df = df.apply(imputer_ligne, axis=1)
    return df

# Appliquer la fonction au dataframe
df = imputer_nb_pieces(df)

# Séparer le DataFrame en deux, un pour "ci.coinafrique.com" et l'autre pour les autres sites
df_coinafrique = df[df['site'] == 'ci.coinafrique.com']
df_others = df[df['site'] != 'ci.coinafrique.com']

# Traitement spécifique pour "ci.coinafrique.com"

# Fonction pour nettoyer les colonnes en supprimant tous les caractères non numériques sauf le point décimal
def clean_numeric_column(column):
    return column.apply(lambda x: re.sub(r'[^\d.]', '', str(x)) if pd.notnull(x) else x)

# Nettoyer les colonnes 'nb_pieces' et 'nb_salle_de_bain'
df['nb_pieces'] = clean_numeric_column(df['nb_pieces'])
df['nb_salle_de_bain'] = clean_numeric_column(df['nb_salle_de_bain'])

# Convertir les colonnes 'nb_pieces' et 'nb_salle_de_bain' en int
df['nb_pieces'] = pd.to_numeric(df_coinafrique['nb_pieces'], errors='coerce').astype('Int64')
df_coinafrique['nb_salle_de_bain'] = pd.to_numeric(df_coinafrique['nb_salle_de_bain'], errors='coerce').astype('Int64')


# Fonction pour extraire la localisation à partir du titre
def extract_localisation(title):
    match = re.search(r'à\s(.+)$', title)
    if match:
        return match.group(1).strip()
    match = re.search(r'-\s(.+)$', title)
    if match:
        return match.group(1).strip()
    return None

# Fonction pour extraire le type de logement à partir du titre
def extract_logement_type(title):
    logement_types = ['appartement', 'maison', 'studio', 'villa', 'chambre', 'bureau', 'penthouse']
    for logement in logement_types:
        if re.search(r'\b' + logement + r'\b', title, re.IGNORECASE):
            return logement
    return None

# Appliquer les fonctions pour extraire la localisation et le type de logement
df_coinafrique['localisation'] = df_coinafrique['title'].apply(extract_localisation)
df_coinafrique["type d'immobilier"] = df_coinafrique['title'].apply(extract_logement_type)


# Fusionner les deux DataFrames
df = pd.concat([df_coinafrique, df_others], ignore_index=True)

# Supprimer les lignes qui ont None dans la variable localisation
df = df.dropna(subset=['localisation'])

# Fonction pour déterminer le type d'immobilier
def classify_immobilier(row):
    text = ' '.join([str(row['title']), str(row['description']), str(row["type d'immobilier"])])
    if re.search(r'duplex', text, re.IGNORECASE):
        return 'maison duplex'
    if re.search(r'appartement|penthouse', text, re.IGNORECASE):
        return 'appartement moderne'
    if re.search(r'maison', text, re.IGNORECASE):
        return 'maison'
    if re.search(r'villa', text, re.IGNORECASE):
        return 'villa moderne'
    if re.search(r'studio', text, re.IGNORECASE):
        return 'studio'
    return 'autre'

# Appliquer la fonction pour créer la colonne 'type_d_immobilier'
df['type_d_immobilier'] = df.apply(classify_immobilier, axis=1)


# Fonction pour déterminer si les toilettes sont internes
def has_toilettes_internes(description):
    return 0 if re.search(r"(toilette|wc|douche|salle d'eau) externe", description, re.IGNORECASE) else 1

# Fonction pour déterminer si la cour est commune
def has_cours_commune(description):
    return 1 if re.search(r"cours commune", description, re.IGNORECASE) else 0

# Appliquer les fonctions pour créer les nouvelles variables
# df['Avec_eau'] = 1
# df['Avec_electricite'] = 1
# df['Materiaux_en_dur'] = 1
df['Toilettes_internes'] = df['description'].apply(has_toilettes_internes)
df['cours_commune'] = df['description'].apply(has_cours_commune)
# df['habitee_par_un_seul_locataire'] = 1


# Définir la liste des quartiers chics
quartiers_chics = ['cocody', 'riviera', 'le plateau', 'zone 4', r'angr(é|e)', 'vallons', 'ambassade', 'golf']

# Fonction pour déterminer si la localisation est dans un quartier chic
def is_quartier_chic(localisation):
    for quartier in quartiers_chics:
        if re.search(quartier, localisation, re.IGNORECASE):
            return 1
    return 0

# Appliquer la fonction pour créer la variable 'quartier_chic'
df['quartier_chic'] = df['localisation'].apply(is_quartier_chic)

# Créer le dictionnaire de mappage
code_mapping = {
    ('maison duplex', 1, 0, 1): "401000006",
    ('maison duplex', 1, 0, 0): "401000006",
    ('appartement moderne', 1, 0, 1): "401000002",
    ('appartement moderne', 1, 0, 0): "401000002",
    ('maison',  1, 0, 1): "401000005",
    ('maison',  1, 0, 0): "401000005",
    ('villa moderne', 1, 0, 1): "401000010",
    ('villa moderne', 1, 0, 0): "401000011",
    ('studio',  1, 0, 1): "401000008",
    ('studio',  1, 0, 0): "401000008",
    ('studio', 0, 1, 0): "401000009"
}

# Fonction pour obtenir le code de classification
def get_classification_code(row, code_mapping):
    key = (
        row['type_d_immobilier'],
        # row['Avec_eau'],
        # row['Avec_electricite'],
        # row['Materiaux_en_dur'],
        row['Toilettes_internes'],
        row['cours_commune'],
        # row['habitee_par_un_seul_locataire'],
        row['quartier_chic']
    )
    code = code_mapping.get(key, None)
    if code is None:
        print(f"Key not found: {key}")
    return code

# Appliquer la fonction pour créer la colonne 'code_var 2014'
df['code_var 2014'] = df.apply(get_classification_code, axis=1, code_mapping=code_mapping)

df.dropna(subset='code_var 2014')


#############################################################


# Définir les dictionnaires de mots regroupés
commodities = {
    'wifi': [r'wifi', r'internet', r'connexion', r'fibre optique'],
    'jardin': [r'jardin(\.locat\d+)?'],
    'meubler': [r'meubl(er|é|e)'],
    'climatisation': [r'climat(isé|isation|iseur)', r'air conditionner', r'split'],
    'garage': [r'garage'],
    'balcon': [r'balcon'],
    'sécuriser': [r'sécur(is|it|is)(er|e|é)'],
    'étage': [r'étage'],
    'parking': [r'parking'],
    'douche': [r'douche'],
    'séjour': [r'séjour', r'sejour'],
    'équiper': [r'équip(er|é|e|ée|ement)'],
    'placard': [r'placard'],
    'salle': [r'salle'],
    'manger': [r'manger'],
    'bureau': [r'bureau'],
    'piscine': [r'piscine'],
    'gardien': [r'gardi(en|en)', r'garde'],
    'électrogène': [r'électrogène'],
    'four': [r'four'],
    'cinéma': [r'cinéma', r'netflix'],
    'ascenseur': [r'ascenseur'],
    'double vitrage': [r'double vitrage'],
    'terrasse': [r'terrasse'],
    'toilette': [r'toilette', r'wc'],
    'buanderie': [r'buanderie'],
    'chauffe': [r'chauff(e|e-eau)'],
    'cuisine': [r'cuisine']
}

natural_externalities = {
    'bord_de_mer': [r'plage', r'mer', r'océan'],
    'forêt': [r'for(ê|e)t'],
    'parc': [r'parc'],
    'montagne': [r'montagne'],
    'rivière': [r'rivi(è|e)re'],
    'lac': [r'lac']
}

artificial_externalities = {
    'zone_commerciale': [r'centre commercial', r'marché', r'zone commerciale'],
    'pharmacie': [r'pharmacie'],
    'restaurant': [r'restaurant', r'resto'],
    'bar': [r'bar', r'maquis'],
    'ambassade': [r'ambassade'],
    'école': [r'écol(e|es)', r'crèch(e|es)', r'lycé(e|es)', r'universit(é|és)'],
    'hôpital': [r'hôpital', r'clinique', r'dispensaire', r'CHU', r'CHR', r'medecin']
}

# Fonction pour extraire les informations de la description
def extract_description_info(df, grouped_words):
    def extract_info(description):
        if not isinstance(description, str):
            return {key: 0 for key in grouped_words}
        
        doc = nlp(description)
        info = {key: 0 for key in grouped_words}
        
        negation = False
        for token in doc:
            if token.dep_ == 'neg':
                negation = True
            for key, patterns in grouped_words.items():
                for pattern in patterns:
                    if re.search(pattern, token.lemma_):
                        info[key] = 0 if negation else 1
                        negation = False
        
        return info
    
    description_info = df['description'].apply(extract_info)
    
    for key in grouped_words.keys():
        df[key] = description_info.apply(lambda x: x[key])
    
    return df

# Appliquer les fonctions pour extraire les informations des commodités et externalités
df = extract_description_info(df, commodities)
df = extract_description_info(df, natural_externalities)
df = extract_description_info(df, artificial_externalities)

# Créer les variables 'commodités', 'externalités_naturelles' et 'externalités_artificielles'
df['commodités'] = df[list(commodities.keys())].max(axis=1)
df['externalités_naturelles'] = df[list(natural_externalities.keys())].max(axis=1)
df['externalités_artificielles'] = df[list(artificial_externalities.keys())].max(axis=1)
#############################################################


# Exporter le dataframe final avec les modifications
import platform
if platform.system() == 'Windows':
    output_path_final = f'D:\\Bureau\\MemoiresStages\\Travaux_techniques\\Traitements\\Datasets\\trying_data_final.csv'
else:
    output_path_final = f'/mnt/d/Bureau/MemoiresStages/Travaux_techniques/Traitements/Datasets/trying_data_final.csv'

df.to_csv(output_path_final, index=False)

print(f"Fichier final avec les modifications sauvegardé sous {output_path_final}")


Les traitements sont en cours, cela pourrait prendre quelques minutes...
Nombre de doublons primaires détectés: 346730
Nombre de doublons secondaires détectés: 976
Fichier après suppression des doublons sauvegardé sous D:\Bureau\MemoiresStages\Travaux_techniques\Traitements\Datasets\trying_data_after_deduplication.csv
Doublons supprimés sauvegardés sous D:\Bureau\MemoiresStages\Travaux_techniques\Traitements\Datasets\trying_data_duplicates.csv


C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_14304\1197112282.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coinafrique['nb_salle_de_bain'] = pd.to_numeric(df_coinafrique['nb_salle_de_bain'], errors='coerce').astype('Int64')
C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_14304\1197112282.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coinafrique['localisation'] = df_coinafrique['title'].apply(extract_localisation)
C:\Users\Wilfried KOMENAN\AppData\Local\Temp\ipykernel_

Fichier final avec les modifications sauvegardé sous D:\Bureau\MemoiresStages\Travaux_techniques\Traitements\Datasets\trying_data_final.csv
